In [1]:
import pandas as pd
import numpy as np

The ipynb file is: the relevant code for establishing the base model of the patient's clinical personal information

# Reading Data

In [ ]:
file_path = r''
df_1 = pd.read_excel(file_path)

In [ ]:
file_path = r''
df_1_GainRatio = pd.read_excel(file_path)

# Feature Selection

In [ ]:
from scipy.stats import entropy

In [ ]:
# 计算整体标签的熵
label_entropy = entropy(df_1['健康状况'].value_counts(normalize=True), base=2)

In [ ]:
# 初始化特征信息增益
feature_information_gains = {}

# 遍历每个特征
for column in df_1.columns[1:]:  # 不包括第一列标签
    # 初始化一个列表，用于存储每个特征值的条件熵
    conditional_entropies = []
    
    # 遍历当前特征的每个唯一值
    for value in df_1[column].unique():
        # 选择当前特征值的子集
        subset = df_1[df_1[column] == value]
        
        # 计算当前特征值在给定标签情况下的条件熵
        conditional_entropy = entropy(subset['健康状况'].value_counts(normalize=True), base=2)
        
        # 计算当前特征值在整体数据集中的权重
        weight = len(subset) / len(df_1)
        
        # 将当前特征值的条件熵加入列表
        conditional_entropies.append(weight * conditional_entropy)
    
    # 计算信息增益，即整体标签熵减去条件熵的加权和
    feature_information_gains[column] = label_entropy - sum(conditional_entropies)

In [ ]:
# 打印每个特征的信息增益
for feature, information_gain in feature_information_gains.items():
    print(f'{feature}: {information_gain}')

In [ ]:
# 获取所有特征的信息增益
information_gains = list(feature_information_gains.values())

In [ ]:
# 计算信息增益的均值
mean_information_gain = sum(information_gains) / len(information_gains)

In [ ]:
# 输出信息增益大于均值的特征名称
selected_features = [feature for feature, information_gain in feature_information_gains.items() if information_gain > mean_information_gain]

print('信息增益大于均值的特征:')
for feature in selected_features:
    print(feature)

In [ ]:
# 初始化存储每个特征的信息增益和分裂信息的字典
feature_information_gains = {}
feature_split_information = {}

In [ ]:
# 遍历每个特征
for column in df_1.columns[1:]:  # 不包括第一列标签
    # 初始化一个列表，用于存储每个特征值的条件熵
    conditional_entropies = []
    
    # 遍历当前特征的每个唯一值
    for value in df_1[column].unique():
        # 选择当前特征值的子集
        subset = df_1[df_1[column] == value]
        
        # 计算当前特征值在给定标签情况下的条件熵
        conditional_entropy = entropy(subset['健康状况'].value_counts(normalize=True), base=2)
        
        # 计算当前特征值在整体数据集中的权重
        weight = len(subset) / len(df_1)
        
        # 将当前特征值的条件熵加入列表
        conditional_entropies.append(weight * conditional_entropy)
    
    # 计算信息增益，即整体标签熵减去条件熵的加权和
    information_gain = label_entropy - sum(conditional_entropies)
    
# ------------------------------------------------------------------------------------------
    
    # 计算分裂信息，即当前特征的熵
    split_information = entropy(df_1[column].value_counts(normalize=True), base=2)
    
    # 计算信息增益率
    information_gain_ratio = information_gain / split_information if split_information != 0 else 0
    
    # 将信息增益率加入字典
    feature_information_gains[column] = information_gain_ratio
    feature_split_information[column] = split_information

In [ ]:
# 打印每个特征的信息增益率
for feature, information_gain_ratio in feature_information_gains.items():
    print(f'{feature}: {information_gain_ratio}')

# Base model establishment

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report

In [ ]:
# 假设 df_1_index 包含特征和标签，其中标签列名为 'target_column'
X = df_1_GainRatio.drop('健康状况', axis=1)
y = df_1_GainRatio['健康状况']

## random_state=20

In [ ]:
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

### RF

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_model = RandomForestClassifier()

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [ ]:
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, scoring='roc_auc', cv=3)
grid_search.fit(X_train, y_train)

In [ ]:
best_rf_model = grid_search.best_estimator_
best_params = grid_search.best_params_

In [ ]:
y_pred = best_rf_model.predict(X_test)
y_prob = best_rf_model.predict_proba(X_test)[:, 1]

# 输出评价指标
auc = roc_auc_score(y_test, y_prob)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("RF：")
print("AUC:", auc)
print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Classification Report:\n", report)

### XGB

In [ ]:
import xgboost as xgb

In [ ]:
xgb_model = xgb.XGBClassifier()

param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

In [ ]:
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='roc_auc', cv=3)
grid_search.fit(X_train, y_train)

In [ ]:
best_xgb_model = grid_search.best_estimator_
best_params = grid_search.best_params_

In [ ]:
y_pred = best_xgb_model.predict(X_test)
y_prob = best_xgb_model.predict_proba(X_test)[:, 1]

auc = roc_auc_score(y_test, y_prob)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("XGBoost：")
print("AUC:", auc)
print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Classification Report:\n", report)

### CAT

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
catboost_model = CatBoostClassifier()

param_grid = {
    'iterations': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'depth': [4, 6, 8],
    'l2_leaf_reg': [1, 3, 5]
}

In [ ]:
grid_search = GridSearchCV(estimator=catboost_model, param_grid=param_grid, scoring='roc_auc', cv=3)
grid_search.fit(X_train, y_train, verbose=0)

In [ ]:
best_catboost_model = grid_search.best_estimator_
best_params = grid_search.best_params_

In [ ]:
y_pred = best_catboost_model.predict(X_test)
y_prob = best_catboost_model.predict_proba(X_test)[:, 1]

auc = roc_auc_score(y_test, y_prob)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("CatBoost：")
print("AUC:", auc)
print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Classification Report:\n", report)

### LGBM

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
lgb_model = LGBMClassifier()

param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

In [ ]:
grid_search = GridSearchCV(estimator=lgb_model, param_grid=param_grid, scoring='roc_auc', cv=3)
grid_search.fit(X_train, y_train)

In [ ]:
best_lgb_model = grid_search.best_estimator_
best_params = grid_search.best_params_

In [ ]:
y_pred = best_lgb_model.predict(X_test)
y_prob = best_lgb_model.predict_proba(X_test)[:, 1]

auc = roc_auc_score(y_test, y_prob)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("LightGBM：")
print("AUC:", auc)
print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Classification Report:\n", report)

## random_state=21

The following parts are basically the same as random_state=20, so they are omitted

## random_state=2

The following parts are basically the same as random_state=20, so they are omitted